In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
# スクレイピング対象のURL
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC005/?fw2=&mt=9999999&cn=9999999&ta=13&et=9999999&sc=13104&shkr1=03&ar=030&bs=040&ct=9999999&shkr3=03&shkr2=03&srch_navi=1&mb=0&shkr4=03&cb=0.0'

# requestsでURLからデータを取得
response = requests.get(url)
response.encoding = response.apparent_encoding

# BeautifulSoupオブジェクトの生成
soup = BeautifulSoup(response.text, 'html.parser')


In [154]:
# 物件情報を格納するためのリスト
properties_list = []

# 空のデータフレームを作成
df = pd.DataFrame()

# 物件情報が含まれる要素をすべて取得
properties = soup.find_all('div', class_='property')

In [155]:
# 物件情報の抽出
for prop in properties:
    # 物件名が記載されているclassを選択
    title = prop.find('h2', class_='property_inner-title').text.strip()
    property_link = prop.find('a', class_='js-cassetLinkHref')['href']

    # 取り扱い店舗が記載されているclassを選択
    property_stores = prop.find('a', class_='js-noCassetteLink').text.strip()
    property_stores_link = prop.find('a', class_='js-noCassetteLink')['href']

    # 住所・アクセス経路・賃料・管理費・間取り・専有面積・向き・築年数・マンションかアパート・敷金・礼金が記載されているtdタグを選択
    detailbox = prop.find('div', class_='detailbox')
    if detailbox:

        # アクセス経路が記載されているtdタグを選択し、改行文字で分割して必要な部分を取得
        access_element = prop.find('div', class_='detailnote-box').text.strip()
        access_1 = access_element.split('\n')[0].strip()
        access_2 =  access_element.split('\n')[1].strip()
        access_3 =  access_element.split('\n')[2].strip()

        # 賃料・管理費が記載されているtdタグを選択し、改行文字で分割して必要な部分を取得
        fee_element = detailbox.find_all('td', class_='detailbox-property-col')[0].text.strip()
        rent_price = fee_element.split('\n')[0].strip()
        management_fee =  fee_element.split('\n')[1].strip()

        # 敷金・礼金が記載されているtdタグを選択し、改行文字で分割して必要な部分を取得
        security_deposit_element = detailbox.find_all('td', class_='detailbox-property-col')[1].text.strip()
        security_deposit = security_deposit_element.split('\n')[0].strip()
        key_money = security_deposit_element.split('\n')[1].strip()

        # 間取り・専有面積・向きが記載されているtdタグを選択し、改行文字で分割して必要な部分を取得
        house_layout_element = detailbox.find_all('td', class_='detailbox-property-col')[2].text.strip()
        house_layout =  house_layout_element.split('\n')[0].strip()
        exclusive_area =  house_layout_element.split('\n')[2].strip()
        direction =  house_layout_element.split('\n')[4].strip()

        # 築年数とマンションかアパートが記載されているtdタグを選択し、改行文字で分割して必要な部分を取得
        building_type_element = detailbox.find_all('td', class_='detailbox-property-col')[3].text.strip()
        building_type =  building_type_element.split('\n')[0].strip()
        building_age = building_type_element.split('\n')[2].strip()

        # 住所が記載されているtdタグを正確に選択
        address = detailbox.find_all('td', class_='detailbox-property-col')[4].text.strip()

    # データリストに情報を追加
    properties_list.append({
        '物件名': title,
        'URL': "https://suumo.jp"+property_link,
        '住所': address,
        'アクセス1': access_1,
        'アクセス2': access_2,
        'アクセス3': access_3,
        '築年数': building_age,
        '賃貸料': rent_price,
        '管理費': management_fee,
        '間取り': house_layout,
        '専有面積': exclusive_area,        
        '向き': direction,
        'タイプ': building_type,
        '敷金': security_deposit,
        '礼金': key_money,
        '取り扱い店舗': property_stores,
        '取り扱い店舗URL': "https://suumo.jp"+property_stores_link
    })

In [156]:
# データフレームの作成
df = pd.DataFrame(properties_list)

In [157]:
# データフレームの表示（省略せずに全て表示する設定も可能です）
df.head()

,物件名,URL,住所,アクセス1,アクセス2,アクセス3,築年数,賃貸料,管理費,間取り,専有面積,向き,タイプ,敷金,礼金,取り扱い店舗,店舗URL
0,Ｍｏｎｋｓｔｏｗｎ夏目坂 104号室,https://suumo.jp/chintai/bc_100375809678/,東京都新宿区戸山１,東京メトロ東西線/早稲田駅 歩4分,都営大江戸線/若松河田駅 歩10分,都営大江戸線/牛込柳町駅 歩12分,新築,28.5万円,管理費 10000円,12LDK,75.09m2,-,マンション,敷28.5万円,礼28.5万円,(株)Street2,https://suumo.jp/chintai/kaisha/kc_030_170304001/
1,東京メトロ東西線 落合駅 14階建 築18年,https://suumo.jp/chintai/bc_100380691923/,東京都新宿区上落合１,東京メトロ東西線/落合駅 歩5分,ＪＲ中央線/東中野駅 歩12分,都営大江戸線/中井駅 歩11分,築18年,15.5万円,管理費 10000円,2LDK,58.68m2,南,マンション,敷15.5万円,礼15.5万円,エールーム 池袋店(株)ライフアシスト,https://suumo.jp/chintai/kaisha/kc_030_160409001/
2,セルクル落合,https://suumo.jp/chintai/bc_100380674198/,東京都新宿区上落合１,東京メトロ東西線/落合駅 歩5分,ＪＲ山手線/高田馬場駅 歩18分,西武新宿線/中井駅 歩11分,築18年,15.5万円,管理費 10000円,2LDK,58.68m2,南,マンション,敷15.5万円,礼15.5万円,アエラス荻窪店 (株)アエラス.FR,https://suumo.jp/chintai/kaisha/kc_030_179833008/
3,セルクル落合,https://suumo.jp/chintai/bc_100380690129/,東京都新宿区上落合１,東京メトロ東西線/落合駅 歩5分,ＪＲ中央線/東中野駅 歩12分,都営大江戸線/中井駅 歩11分,築18年,15.5万円,管理費 10000円,2LDK,58.68m2,南,マンション,敷15.5万円,礼15.5万円,(株)アクセス新宿店,https://suumo.jp/chintai/kaisha/kc_030_144590001/
4,東京メトロ東西線 落合駅 14階建 築18年,https://suumo.jp/chintai/bc_100380740603/,東京都新宿区上落合１,東京メトロ東西線/落合駅 歩5分,ＪＲ中央線/東中野駅 歩12分,都営大江戸線/中井駅 歩11分,築18年,15.5万円,管理費 10000円,2LDK,58.68m2,南,マンション,敷15.5万円,礼15.5万円,(株)エールーム新宿店,https://suumo.jp/chintai/kaisha/kc_030_138683001/


In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   物件名     30 non-null     object
 1   URL     30 non-null     object
 2   住所      30 non-null     object
 3   アクセス1   30 non-null     object
 4   アクセス2   30 non-null     object
 5   アクセス3   30 non-null     object
 6   築年数     30 non-null     object
 7   賃貸料     30 non-null     object
 8   管理費     30 non-null     object
 9   間取り     30 non-null     object
 10  専有面積    30 non-null     object
 11  向き      30 non-null     object
 12  タイプ     30 non-null     object
 13  敷金      30 non-null     object
 14  礼金      30 non-null     object
 15  取り扱い店舗  30 non-null     object
 16  店舗URL   30 non-null     object
dtypes: object(17)
memory usage: 4.1+ KB
